In [1]:
import torch
import torch.optim as optim
from src.pytorch_efficientnet import EfficientNetwork, augment, augmented_pred
from src.train_test_functions import train_model
import json
import numpy as np
from src.torch_dataset import ZootrDataset 
import os
import pandas as pd
from sklearn.metrics import roc_auc_score

## Set constants

In [2]:
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
EPOCHS = 10

In [3]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']

In [4]:
load_data_objects = False
save_data_objects = True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set up data loaders

In [6]:
if  load_data_objects:
    torch.load('Models/train_data.pth')
    torch.load('Models/valid_data.pth')
else:
    items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
    #Get all items
    non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'Models/train_data.pth')
        torch.save(valid_data, 'Models/valid_data.pth')

In [7]:
data = {'train': all_images[0:int(np.ceil(len(all_images)*0.7))], 
        'val': all_images[int(np.ceil(len(all_images)*0.7)):int(np.ceil(len(all_images)*0.8))], 
        'test': all_images[int(np.ceil(len(all_images)*0.8)):]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': int(np.ceil(len(all_images)*0.7)), 'val': int(np.ceil(len(all_images)*0.8)) - int(np.ceil(len(all_images)*0.7)), 
                 'test': len(all_images) - int(np.ceil(len(all_images)*0.8))}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}

dics = {
    'dataloaders': dataloaders,
    'dataset_sizes': dataset_sizes
}

## Model

In [8]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

Loaded pretrained weights for efficientnet-b2


In [9]:
mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)

Epoch 1


100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


Train loss: 0.4158954988616701 Val loss: 0.2724017857929359
Epoch 2


100%|██████████| 10/10 [00:06<00:00,  1.50it/s]


Train loss: 0.1514995639787185 Val loss: 0.17028846962440417
Epoch 3


...

Train loss: 0.0012402670456671929 Val loss: 0.10478650847368914
Epoch 24


100%|██████████| 10/10 [00:06<00:00,  1.47it/s]


Train loss: 0.0007549111345417942 Val loss: 0.10356637283682751
Epoch 25


100%|██████████| 10/10 [00:06<00:00,  1.57it/s]

Train loss: 0.0009422450578482223 Val loss: 0.08818363119591621
Training complete in 23m 52s
Best val loss: 0.000755


## Get test metrics

In [10]:
test_loader = dataloaders['test']

all_preds = torch.empty(0, 2)
all_labels = []
for imgs, label in test_loader:
    imgs_pred = mdl(imgs)

    all_preds = torch.cat((all_preds, imgs_pred))
    all_labels += label
labels = [i.item() for i in all_labels]
#Get all labels

In [11]:
preds_prob = torch.softmax(all_preds, dim=1)
preds_prob = pd.DataFrame(preds_prob.detach().numpy())
preds_prob.columns = ['False', 'True']
preds_prob['Label'] = labels

In [12]:
preds_prob.sort_values('True', ascending=False)

,False,True,Label
149,0.000034,9.999660e-01,1
168,0.000051,9.999491e-01,1
303,0.000065,9.999348e-01,1
258,0.000092,9.999082e-01,1
222,0.000113,9.998869e-01,1
...,...,...,...
71,0.999992,7.793835e-06,0
188,0.999993,7.189923e-06,0
38,0.999994,6.192588e-06,0
131,0.999998,1.910734e-06,0


In [13]:
roc_auc_score(preds_prob['Label'], preds_prob['True'])

0.9525915008597396

In [14]:
1/0

ZeroDivisionError: division by zero

## MLflow

In [1]:
import mlflow

In [17]:
import subprocess
import time
import webbrowser


def start_mlflow(timeout=5):
    subprocess.Popen(
        ['mlflow', 'ui', '--backend-store-uri', 'file:Tracking'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    time.sleep(3)

    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("check-localhost-connection")
    webbrowser.open("http://localhost:5000")
    


In [18]:
start_mlflow()

In [2]:
!mlflow ui --backend-store-uri file:Tracking
#Needs to be interrupted

^C


In [3]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("check-localhost-connection")

<Experiment: artifact_location='mlflow-artifacts:/664169674059468358', creation_time=1755030492211, experiment_id='664169674059468358', last_update_time=1755030492211, lifecycle_stage='active', name='check-localhost-connection', tags={}>

In [4]:
NR_EPOCHS=5

with mlflow.start_run() as run:
  params = {
      "num_epochs": NR_EPOCHS,
      "scheduler": scheduler.__dict__,
      "optimizer": optimizer
  }
  # Log training parameters.
  mlflow.log_params(params)

  # Log model summary.
  #with open("model_summary.txt", "w") as f:
  #    f.write(str(summary(model)))
  #mlflow.log_artifact("model_summary.txt")

  model = EfficientNetwork(output_size=2, b1=False, b2=True)
  mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=NR_EPOCHS)
  

  # Save the trained model to MLflow.
  model_info = mlflow.pytorch.log_model(mdl, name="first_model")

  all_preds = torch.empty(0, 2)
  all_labels = []
  for imgs, label in test_loader:
      imgs_pred = mdl(imgs)

      all_preds = torch.cat((all_preds, imgs_pred))
      all_labels += label
  labels = [i.item() for i in all_labels]

  preds_prob = torch.softmax(all_preds, dim=1)
  preds_prob = pd.DataFrame(preds_prob.detach().numpy())
  preds_prob.columns = ['False', 'True']
  preds_prob['Label'] = labels

  test_loss = criterion(all_preds, torch.tensor(labels)).item()
  test_roc = roc_auc_score(preds_prob['Label'], preds_prob['True'])

  mlflow.log_metrics({"test_loss": test_loss, "test_roc": test_roc})

🏃 View run rare-pig-840 at: http://localhost:5000/#/experiments/664169674059468358/runs/c120bb1186c04508a232f957dd7e0248
🧪 View experiment at: http://localhost:5000/#/experiments/664169674059468358


NameError: name 'scheduler' is not defined

In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)